In [1]:
import sys
# sys.path.append("/Users/user/question-retrieval-KIPerWeb/")
from utils import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import json
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
from ranx import Qrels, evaluate, Run, compare
import swifter
import json
import requests
from sklearn.preprocessing import LabelEncoder
from trectools import TrecPoolMaker, TrecRun
from tqdm import tqdm
from collections import Counter
import spacy
from io import StringIO

## Data and indices preparation

In [2]:
# # Read preprocessed data
# qb = pd.read_csv("/Users/user/EdTec-QBuilder/data/question_bank_full.csv", index=False).fillna("N/A")
# qb = qb.drop(columns=['docid'])
# qb.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
# qb['docid'] = qb.index
# qb['content'] = qb['content'].apply(lambda x:remove_stop_words(x, 'german'))


# # Preprocessing
# # Models for preprocessing
# nlp = spacy.load("de_core_news_sm")
# tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

# qb['content'] = qb['content'].apply(lambda x : preprocess_documents(x, tagger_de))
# qb

In [3]:
# demo_qb = qb.query("source=='ChatGPT3.5'")
# demo_qb.rename({"irt_difficulty_label":"level_difficulty"}, inplace = True, axis=1)
# demo_qb.to_csv("/Users/user/EdTec-QBuilder/data/question_bank_demo.csv")
# demo_qb

In [4]:
# model_14 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-cosine")
# index_14 = create_index(demo_qb, model_14, index_path="/Users/user/EdTec-QBuilder/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")



### Re ranker

In [5]:
df = pd.read_csv("/Users/user/EdTec-QBuilder/data/question_bank_demo.csv").fillna("N/A").drop(columns=['Unnamed: 0'])
df

,answer_type_id,answer_type_name,answers,frontend_info_text,points,question_type_id,question_type_name,onlinetest_title,text,variable,...,lang,topic_label_en,topic_label_de,source,answers_content,related_topics,topic_label_de_fixed,irt_difficulty,level_difficulty,docid
0,4,Single Choice,"[{'Mechanische Energie': 'false', 'Chemische R...",N/A,N/A,N/A,N/A,N/A,Was ist die Grundlage der Elektrotechnik?,N/A,...,GERMAN,N/A,Elektrotechnik,ChatGPT3.5,Mechanische Energie Chemische Reaktionen Elekt...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",elektrotechnik,N/A,N/A,2812
1,4,Single Choice,"[{'Ein Computerprogramm': 'false', 'Eine Kombi...",N/A,N/A,N/A,N/A,N/A,Was ist eine elektrische Schaltung?,N/A,...,GERMAN,N/A,Elektrische Schaltungen,ChatGPT3.5,Ein Computerprogramm Eine Kombination von Scha...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",elektrotechnik,N/A,N/A,2813
2,4,Single Choice,"[{'U = R * I': 'true', 'F = m * a': 'false', '...",N/A,N/A,N/A,N/A,N/A,Wie lautet das Ohmsche Gesetz?,N/A,...,GERMAN,N/A,Ohmsches Gesetz,ChatGPT3.5,U = R * I F = m * a E = mc^2 P = V * I,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",elektrotechnik,N/A,N/A,2814
3,4,Single Choice,"[{'Nur um Stromrechnungen zu senken': 'false',...",N/A,N/A,N/A,N/A,N/A,Warum ist die elektrische Sicherheit wichtig?,N/A,...,GERMAN,N/A,Elektrische Sicherheit,ChatGPT3.5,Nur um Stromrechnungen zu senken Um elektrisch...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",elektrotechnik,N/A,N/A,2815
4,4,Single Choice,"[{'Klare Botschaft und Struktur': 'true', 'Lan...",N/A,N/A,N/A,N/A,N/A,Was ist ein wichtiger Aspekt einer erfolgreich...,N/A,...,GERMAN,N/A,Präsentationstechniken,ChatGPT3.5,Klare Botschaft und Struktur Lange und kompliz...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",visualisierungs- und präsentationstechniken,N/A,N/A,2816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2807,4,Single Choice,[{'Antwort': 'Ein Kuchendiagramm ist eine Dars...,N/A,N/A,N/A,N/A,N/A,Was ist ein Kuchendiagramm?,N/A,...,GERMAN,N/A,Visualisierungstechniken,ChatGPT3.5,Ein Kuchendiagramm ist eine Darstellung von Da...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",visualisierungs- und präsentationstechniken,N/A,N/A,5619
2808,4,Single Choice,[{'Antwort': 'Ein Balken-Diagramm ist eine Dar...,N/A,N/A,N/A,N/A,N/A,Was ist ein Balken-Diagramm?,N/A,...,GERMAN,N/A,Visualisierungstechniken,ChatGPT3.5,Ein Balken-Diagramm ist eine Darstellung von D...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",visualisierungs- und präsentationstechniken,N/A,N/A,5620
2809,4,Single Choice,[{'Antwort': 'Ein Boxplot ist eine Darstellung...,N/A,N/A,N/A,N/A,N/A,Was ist ein Boxplot?,N/A,...,GERMAN,N/A,Visualisierungstechniken,ChatGPT3.5,Ein Boxplot ist eine Darstellung von Daten in ...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",visualisierungs- und präsentationstechniken,N/A,N/A,5621
2810,4,Single Choice,[{'Antwort': 'Ein Netzwerkdiagramm ist eine Da...,N/A,N/A,N/A,N/A,N/A,Was ist ein Netzwerkdiagramm?,N/A,...,GERMAN,N/A,Visualisierungstechniken,ChatGPT3.5,Ein Netzwerkdiagramm ist eine Darstellung von ...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",visualisierungs- und präsentationstechniken,N/A,N/A,5622


In [6]:
import pandas as pd
from sentence_transformers import CrossEncoder

def re_rank_dataframe(df, query, model):
    query_doc_pairs = [(query, doc) for doc in df['content']]

    scores = model.predict(query_doc_pairs)

    if len(scores.shape) == 2 and scores.shape[1] == 2:
        scores = scores[:, 0]

    df['score'] = scores
    df_sorted = df.sort_values(by='score', ascending=True)

    return df_sorted


# Example usage
query = "garten"
# re_ranking_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
re_ranking_model = CrossEncoder("amberoad/bert-multilingual-passage-reranking-msmarco")

In [7]:
re_rank_dataframe(df, query, re_ranking_model)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,answer_type_id,answer_type_name,answers,frontend_info_text,points,question_type_id,question_type_name,onlinetest_title,text,variable,...,topic_label_en,topic_label_de,source,answers_content,related_topics,topic_label_de_fixed,irt_difficulty,level_difficulty,docid,score
1901,5,Multiple Choice,"[{'Antwort': 'Reichlich Blumen.', 'ist richtig...",N/A,N/A,N/A,N/A,N/A,Was ist ein typisches Merkmal eines Zen-Gartens?,N/A,...,N/A,Garten- und Landschaftsbau,ChatGPT3.5,Reichlich Blumen. Rasenflächen. Bunte Skulptur...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",garten- und landschaftsbau,N/A,N/A,4713,3.018040
19,4,Single Choice,"[{'Rasenmäher': 'false', 'Spaten': 'false', 'H...",N/A,N/A,N/A,N/A,N/A,"Welches Gartengerät wird verwendet, um den Bod...",N/A,...,N/A,Gartengeräte,ChatGPT3.5,Rasenmäher Spaten Hacke Gießkanne,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",garten- und landschaftsbau,N/A,N/A,2831,3.198225
1905,5,Multiple Choice,"[{'Antwort': 'Reichlich Bäume und Sträucher.',...",N/A,N/A,N/A,N/A,N/A,Was ist ein typisches Merkmal eines Steingartens?,N/A,...,N/A,Garten- und Landschaftsbau,ChatGPT3.5,Reichlich Bäume und Sträucher. Grasbedeckung. ...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",garten- und landschaftsbau,N/A,N/A,4717,3.297917
1890,5,Multiple Choice,"[{'Antwort': 'Lavendel (Lavandula).', 'ist ric...",N/A,N/A,N/A,N/A,N/A,Welche der folgenden Pflanzen ist ein immergrü...,N/A,...,N/A,Garten- und Landschaftsbau,ChatGPT3.5,Lavendel (Lavandula). Rosenstrauch (Rosa). Eib...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",garten- und landschaftsbau,N/A,N/A,4702,3.546151
1897,5,Multiple Choice,"[{'Antwort': 'Goldfische.', 'ist richtig': Fal...",N/A,N/A,N/A,N/A,N/A,Was ist ein typischer Bestandteil eines Garten...,N/A,...,N/A,Garten- und Landschaftsbau,ChatGPT3.5,Goldfische. Papageienfische. Seerosen. Wasserl...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",garten- und landschaftsbau,N/A,N/A,4709,3.572113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,5,Multiple Choice,[{'Antwort': 'Die Geschäftsleitung hat keine V...,N/A,N/A,N/A,N/A,N/A,Welche Verantwortung trägt die Geschäftsleitun...,N/A,...,N/A,Management-Kontrollsystem,ChatGPT3.5,Die Geschäftsleitung hat keine Verantwortung i...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",management-kontrollsystem,N/A,N/A,3554,5.833334
1730,5,Multiple Choice,[{'Antworttext': 'Die Weltgesundheitsorganisat...,N/A,N/A,N/A,N/A,N/A,Welche Organisationen sind Teil der Vereinten ...,N/A,...,N/A,Auswärtiges Amt,ChatGPT3.5,[{'Antworttext': 'Die Weltgesundheitsorganisat...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",diplomatie und außenpolitische verwaltung,N/A,N/A,4542,5.837636
2670,4,Single Choice,[{'Antwort': 'Der Technologie- und IT-Sektor h...,N/A,N/A,N/A,N/A,N/A,Welcher Wirtschaftssektor hat tendenziell eine...,N/A,...,N/A,Frauen in der Belegschaft,ChatGPT3.5,Der Technologie- und IT-Sektor hat tendenziell...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",frauen in der belegschaft,N/A,N/A,5482,5.837964
2463,5,Multiple Choice,[{'Antwort': 'Die Berichterstattung hat keine ...,N/A,N/A,N/A,N/A,N/A,Welche Rolle spielt die Berichterstattung in e...,N/A,...,N/A,Management-Kontrollsystem,ChatGPT3.5,Die Berichterstattung hat keine Rolle in einem...,"[{'topic': 'Allgemeinwissen', 'rate': 0.316226...",management-kontrollsystem,N/A,N/A,5275,5.859292
